# Scrape through API provided by reddit

In [ ]:
pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 1.4 MB/s eta 0:00:00


In [ ]:
import praw

reddit = praw.Reddit(
    client_id='0vmP-iHQB4ZjA85Pfsftqw',
    client_secret='Ej3QuV8Q51USLILrTjKYFAU3WdH0tQ',
    user_agent='Mozilla/5.0',
    user_name='Sufficient_Bonus_515',
    check_for_async=False
)


In [ ]:
# Access the teslamotors subreddit
subreddit = reddit.subreddit('teslamotors')

# Initialize an empty set to store unique titles
unique_titles = set()

# Loop through the top 1000 hot posts
for post in subreddit.hot(limit=1000):
    # Check if the title is not in the set of unique titles
    if post.title not in unique_titles:
        # Add the title to the set of unique titles
        unique_titles.add(post.title)

# Convert the set of unique titles back to a list
unique_titles_list = list(unique_titles)

Praw has limit that we can only get 1000 response. No loop works here.

In [ ]:
print(len(unique_titles_list))

719


Only receive 719 in the end

In [ ]:
# Save the titles to a text file
with open('unique_hot_titles_list.txt', 'w', encoding='utf-8') as file:
    # Loop through the titles and write each title to the file
    for title in unique_titles_list:
        file.write(title + '\n')

# Sentiment Analysis

In [ ]:
titles = []

# read the titles
with open("unique_hot_titles_list.txt", "r", encoding="utf-8") as file:
    for line in file:
        titles.append(line.strip())

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

## analyze each title

In [ ]:
# Initialize SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Initialize sentiment score
sentiment_scores = []

for title in titles:
    # get the score for each title
    score = sia.polarity_scores(title)
    sentiment_scores.append(score)

for i, score in enumerate(sentiment_scores):
    print(f"title {i+1} 's score': {score}")

title 1 's score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
title 2 's score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
title 3 's score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
title 4 's score': {'neg': 0.5, 'neu': 0.5, 'pos': 0.0, 'compound': -0.8402}
title 5 's score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
title 6 's score': {'neg': 0.231, 'neu': 0.769, 'pos': 0.0, 'compound': -0.5574}
title 7 's score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
title 8 's score': {'neg': 0.297, 'neu': 0.586, 'pos': 0.117, 'compound': -0.4588}
title 9 's score': {'neg': 0.238, 'neu': 0.762, 'pos': 0.0, 'compound': -0.3612}
title 10 's score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
title 11 's score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
title 12 's score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
title 13 's score': {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}
title 14 's score

neg:negative, neu:Neutral,pos:positive
compound means what is the sentiment for this title, positive number means positive

## analyze total

In [ ]:
# initialize
total_sentiment_score = {'neg': 0.0, 'neu': 0.0, 'pos': 0.0, 'compound': 0.0}

for title in titles:
    # get score for each title
    score = sia.polarity_scores(title)
    # cumulate each score
    for key in total_sentiment_score:
        total_sentiment_score[key] += score[key]

# get the avg score
num_titles = len(titles)
for key in total_sentiment_score:
    total_sentiment_score[key] /= num_titles

# print the result
print("total:", total_sentiment_score)

total: {'neg': 0.04686926286509038, 'neu': 0.8775424200278169, 'pos': 0.07558553546592493, 'compound': 0.05085159944367175}


the attitude towards teslamotors is Neutral

# analyze common topic

(something about the code:
1. why use LDA here: because worldcloud can not generate 'topic'. LDA is widely used in here;
2. how to determine the stop word: I first run without any stop word to see the result, and choose the one without meaning as stop word(like as in for...)
3. how to determine 5 topic: just try a lot of number see which one makes more sense
4. the use of plot: click one topic and can see the key words accordingly)


In [ ]:
pip install pyLDAvis

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 33.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.4/345.4 kB 14.8 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 0.25.0 requires pandas<2.1.4,>=1.5.0, but you have pandas 2.2.1 which is incompatible.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.2.1 which is incompatible.


In [ ]:
import pyLDAvis.gensim_models
from gensim.models import LdaModel
from gensim import corpora
from gensim.utils import simple_preprocess

# Define custom_stopwords and other functions here
custom_stopwords = set(['the', 'is', 'and', 'tesla', 'to', 'for', 'in', 'on', 'of', 'with', 'new', 'by'])

def preprocess_titles(titles):
    processed_titles = []
    for title in titles:
        processed_title = [word for word in simple_preprocess(title, deacc=True) if word not in custom_stopwords]
        processed_titles.append(processed_title)
    return processed_titles

def create_corpus(texts):
    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]
    return dictionary, corpus

def run_lda(corpus, dictionary, num_topics=5):
    lda_model = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics)
    return lda_model

def print_topics(lda_model):
    for topic in lda_model.print_topics():
        print(topic)

def main(titles):
    processed_titles = preprocess_titles(titles)
    dictionary, corpus = create_corpus(processed_titles)
    lda_model = run_lda(corpus, dictionary)
    print_topics(lda_model)
    return lda_model, corpus, dictionary

# Call the main function to get lda_model, corpus, and dictionary
if __name__ == "__main__":
    titles = []
    with open("unique_hot_titles_list.txt", "r", encoding="utf-8") as file:
        for line in file:
            titles.append(line.strip())
    lda_model, corpus, dictionary = main(titles)

# Visualize the LDA model
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, dictionary)
vis

(0, '0.019*"cybertruck" + 0.019*"model" + 0.006*"as" + 0.006*"now" + 0.006*"it" + 0.005*"car" + 0.005*"back" + 0.005*"are" + 0.004*"from" + 0.004*"that"')
(1, '0.030*"model" + 0.030*"cybertruck" + 0.007*"no" + 0.007*"performance" + 0.007*"at" + 0.006*"highland" + 0.006*"video" + 0.005*"this" + 0.005*"now" + 0.005*"us"')
(2, '0.011*"fsd" + 0.011*"cybertruck" + 0.011*"model" + 0.010*"update" + 0.008*"as" + 0.007*"beta" + 0.007*"out" + 0.006*"has" + 0.006*"after" + 0.006*"berlin"')
(3, '0.021*"cybertruck" + 0.016*"model" + 0.007*"at" + 0.006*"berlin" + 0.005*"it" + 0.005*"giga" + 0.005*"spotted" + 0.005*"over" + 0.004*"range" + 0.004*"that"')
(4, '0.031*"cybertruck" + 0.012*"model" + 0.009*"at" + 0.007*"elon" + 0.007*"musk" + 0.006*"nacs" + 0.006*"range" + 0.006*"will" + 0.006*"from" + 0.005*"fsd"')


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.038574 -0.071547       1        1  23.991302
2      0.103873 -0.031842       2        1  19.515417
1     -0.046814  0.064172       3        1  19.323943
4      0.029494  0.070133       4        1  18.834444
3     -0.047978 -0.030916       5        1  18.334893, topic_info=           Term        Freq       Total Category  logprob  loglift
45          fsd   24.000000   24.000000  Default  30.0000  30.0000
147      berlin   13.000000   13.000000  Default  29.0000  29.0000
7    cybertruck  132.000000  132.000000  Default  28.0000  28.0000
457        giga   13.000000   13.000000  Default  27.0000  27.0000
86        model  105.000000  105.000000  Default  26.0000  26.0000
..          ...         ...         ...      ...      ...      ...
72           be    3.694622   15.051856   Topic5  -5.6939   0.2917
532          up    3.334247   13.883209   Topic5  -5.7966   0.2699
136         has    3.510350   21.806374   Topic5  -5.7451  -0.1301
273        from    3.475607   23.905306   Topic5  -5.7550  -0.2320
138          no    3.042095   21.001986   Topic5  -5.8883  -0.2357

[349 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
891       4  0.469471       able
915       5  0.941645  according
549       3  0.401021     action
549       4  0.601531     action
235       1  0.153561       adds
...     ...       ...        ...
160       2  0.143713        you
160       3  0.215569        you
160       4  0.143713        you
160       5  0.071856        you
261       5  0.941410       zero

[591 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2, 5, 4])

Interpret:
- Topic 1:
Keywords: cybertruck, model, as, now, it, car, back, are, from, that
This topic seems to be about the Cybertruck and Tesla's model. It might discuss the current status or availability of the Cybertruck, as well as general aspects related to Tesla's models.
- Topic 2:
Keywords: model, cybertruck, no, performance, at, highland, video, this, now, us
This topic appears to focus on the Tesla models and the Cybertruck, possibly discussing their performance, features, and events like highland testing or video releases.
- Topic 3:
Keywords: fsd, cybertruck, model, update, as, beta, out, has, after, berlin
This topic seems to revolve around updates, especially related to the Full Self-Driving (FSD) feature, as well as developments concerning the Cybertruck, possibly mentioning beta releases and events in Berlin.
- Topic 4:
Keywords: cybertruck, model, at, berlin, it, giga, spotted, over, range, that
This topic may involve discussions about the Cybertruck and Tesla's model, possibly focusing on events or developments in Berlin, as well as mentions of the vehicle's range and sightings. This looks similar as Topic 3.
- Topic 5:
Keywords: cybertruck, model, at, elon, musk, nacs, range, will, from, fsd
This topic could relate to the Cybertruck and Tesla's model, potentially discussing announcements or statements made by Elon Musk, as well as mentions of events like NACS (North American Cybertruck Show) and updates on range and FSD.